In [1]:
import pandas as pd
import numpy as np
import time
import math

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
vic_data = pd.read_csv('../../team_code/chieh/victoria.csv', parse_dates=[0])

In [3]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1

TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

CHARGE = -1
DISCHARGE = 1

In [4]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [5]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    df['decision'] = 0
    
    df = df.drop([0], axis=0) 
    
    return df

In [6]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'decision'] = CHARGE

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'decision'] = DISCHARGE

    return df

In [7]:
POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """
    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap



def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch



def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
    
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue



def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if(df.at[i,'decision'] == CHARGE):
        raw_power = -min(BATTERY_POWER, (BATTERY_CAPACITY-opening_cap)/EFFICIENCY*2)
    if(df.at[i,'decision'] == DISCHARGE):
        raw_power = min(BATTERY_POWER, opening_cap/EFFICIENCY*2)
            
    return raw_power



def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df



def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

In [8]:
def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    # Find Optimal Charging and Discharging period
    df3 = algorithm3(df2)
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [9]:
# Highest Revenue using cross validation
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
vic_price = run_all(vic_data)

Total revenue in the dataset: 122330144
Total days in the dataset: 1322.0
Revenue per day: 92534.1482602118
Time Complexity for running the entire Algorithm 3: 20.853508710861206s


In [10]:
vic_price.head(30)

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,decision
1,2018-01-01 00:30:00,92.46,0,0,0,0,0,1
2,2018-01-01 01:00:00,87.62,0,0,0,0,0,1
3,2018-01-01 01:30:00,73.08,0,0,0,0,0,1
4,2018-01-01 02:00:00,70.18,0,0,0,0,0,1
5,2018-01-01 02:30:00,67.43,0,0,0,0,0,1
6,2018-01-01 03:00:00,66.31,0,0,0,0,0,0
7,2018-01-01 03:30:00,67.72,0,0,0,0,0,1
8,2018-01-01 04:00:00,65.50,0,0,0,0,0,0
9,2018-01-01 04:30:00,64.50,-300,-150,-9763,0,135,-1
10,2018-01-01 05:00:00,65.41,-300,-150,-9901,135,270,-1


# Linear Programming Example

- pip install PuLP
- reference : https://towardsdatascience.com/maximizing-profit-using-linear-programming-in-python-642520c43f6

In [11]:
# import the library PuLP as p 
import pulp as p

In [12]:
# Set Up a LP Maximization Problem:
Lp_prob = p.LpProblem('Activity-Analysis_1', p.LpMaximize) # Here we named the Problem "Acitity-Analysis_1".

In [13]:
# Set Up Problem Variables: 
c = p.LpVariable("c", lowBound = 0) # "c" for chair
t = p.LpVariable("t", lowBound = 0) # "t" for table
d = p.LpVariable("d", lowBound = 0) # "d" for desk
b = p.LpVariable("b", lowBound = 0) # "b" for bookcase

In [14]:
# Create Objective Function:
Lp_prob += 45 * c + 80 * t + 110 * d + 55 * b 

In [15]:
# Create Constraints: 
Lp_prob += 5 * c + 20 * t + 15 * d + 22 * b <= 20000
Lp_prob += 10 * c + 15 * t + 25 * d + 20 * b <= 4000
Lp_prob += 3 * c + 8 * t + 15 * d + 10 * b <= 2000
Lp_prob += 4 * c + 20 * d <= 3000
Lp_prob += 20 * b <= 500

In [16]:
# Show the problem:
print(Lp_prob) # note that it's shown in alphabetical order

Activity-Analysis_1:
MAXIMIZE
55*b + 45*c + 110*d + 80*t + 0
SUBJECT TO
_C1: 22 b + 5 c + 15 d + 20 t <= 20000

_C2: 20 b + 10 c + 25 d + 15 t <= 4000

_C3: 10 b + 3 c + 15 d + 8 t <= 2000

_C4: 4 c + 20 d <= 3000

_C5: 20 b <= 500

VARIABLES
b Continuous
c Continuous
d Continuous
t Continuous



In [17]:
### Simplifying the Problem and Solving it ###

In [18]:
# Generate a New LP Maximization Problem:
Lp_prob2 = p.LpProblem('Activity-Analysis_2', p.LpMaximize)

In [19]:
# Generate Problem Variables (>= 0): 
c = p.LpVariable("c", lowBound = 0)
t = p.LpVariable("t", lowBound = 0)


In [20]:
# Create Objective Function:
Lp_prob2 += 45 * c + 80 * t #+ 110 * d + 55 * b 

In [21]:
# Set Up the Constraints: 
Lp_prob2 += 5 * c + 20 * t <= 400
Lp_prob2 += 10 * c + 15 * t <= 450

In [22]:
# Show the problem:
print(Lp_prob2) # note that it's shown in alphabetical order

Activity-Analysis_2:
MAXIMIZE
45*c + 80*t + 0
SUBJECT TO
_C1: 5 c + 20 t <= 400

_C2: 10 c + 15 t <= 450

VARIABLES
c Continuous
t Continuous



In [23]:
# Solve the Problem:
status = Lp_prob2.solve()
print(p.LpStatus[status])   # Display Solution Status

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/student.unimelb.edu.au/hestertzehun/.local/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/7e35be8a313246a69705b721021327ef-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/7e35be8a313246a69705b721021327ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 14 RHS
At line 17 BOUNDS
At line 18 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj -0 Dual inf 147.5 (2)
0  Obj -0 Dual inf 147.5 (2)
2  Obj 2200
Optimal - objective value 2200
Optimal objective 2200 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Optimal


In [24]:
# Printing the final solution 
print(p.value(c), p.value(t), p.value(Lp_prob2.objective))
# Result: 24.0 14.0 2200.0

24.0 14.0 2200.0


# Linear Programming - Attempt

In [25]:
# Generate a New LP Maximization Problem:
#Lp_attempt = p.LpProblem('Attempt-1', p.LpMaximize)

In [26]:
# Set Up Problem Variables: 

#price = p.LpVariable("price", lowBound = 0) # "spot price"
#raw_power = p.LpVariable("raw_power") # "spot price"

In [27]:
# Create Objective Function:
#Lp_attempt += price * 300 - (-300) * price

Failed : Non-constant expression

# Forward DP Algorithm - Attempt

Reference : https://www.osti.gov/servlets/purl/1489128

In [29]:
# Define the state space : Possible State of Battery Capacity
opening_cap = 0
dispatch = 0
eff = 0.9
math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))

0